In [1]:
from model.rnn import GRUDecoder
from model.autoencoder import AutoEncoder, ConvEncoderLayer
import torch        

In [2]:
def get_device():
    # Check if CUDA is available
    if torch.cuda.is_available():
        # If CUDA is available, select the first CUDA device
        device = torch.device("cuda:0")
        print("Using CUDA device:", torch.cuda.get_device_name(0))
    # Check for MPS availability on supported macOS devices (requires PyTorch 1.12 or newer)
    elif torch.backends.mps.is_available():
        # If MPS is available, use MPS device
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders) device")
    else:
        # Fallback to CPU if neither CUDA nor MPS is available
        device = torch.device("cpu")
        print("Using CPU")
    return device


In [3]:
x = torch.rand((2,10,6))

model = ConvEncoderLayer(input_size=10, output_size=4)

In [4]:
model(x)

TypeError: forward() missing 1 required positional argument: 'input'

In [4]:
AutoEncoder(vocab_size=10, embedding_size=96, encoding_size=16, num_layers=3).decoder

ModuleList(
  (0): AutoEncoderLayer(
    (linear): Linear(in_features=16, out_features=24, bias=True)
    (act): Mish()
    (norm): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (1): AutoEncoderLayer(
    (linear): Linear(in_features=24, out_features=48, bias=True)
    (act): Mish()
    (norm): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
  )
  (2): AutoEncoderLayer(
    (linear): Linear(in_features=48, out_features=96, bias=True)
    (act): Mish()
  )
)

In [3]:
## 
# input_size -> the size of the embedding of the autoencoder model
# hidden_size -> the size of the RNN to use in the decoder (the input_size and hidden_size can be different)
model = GRUDecoder(input_size=6, hidden_size=10, max_seq_len=4).to(get_device())

Using MPS (Metal Performance Shaders) device


In [4]:
# This is just an example

MAX_SEQ_LEN = 4 # max number of surveyas a person (in our dataset can have)
INPUT_SIZE = 6 # hidden dimmensions of autoencodder.

# let's say we have a person who only have 2 surveys
x0 = torch.rand(INPUT_SIZE) # embedding for the 1st survey 
x1 = torch.rand(INPUT_SIZE) # embedding for the 2nd survey

# the tensor for the person should be on the shape [MAX_SEQ_LEN, INPUT_SIZE]

e = torch.zeros(MAX_SEQ_LEN, INPUT_SIZE)
e[0] = x0
e[1] = x1
e = e.to(get_device()) # so this is a tensor for the person
#we also need to specify that the sequence has 'empty' embeddings
mask = torch.BoolTensor([True, True, False, False]).to(get_device()) # the last two dimensions are empty
## it is important that you append existing survey embeddings right next to each other (even if the year is missign between them, they should be still appended one after another)

## let assume we have a batch of people, I am reusing the same person, but in the pipeline is should be different people
# the batch size is 3 here 

x = torch.stack([e,e,e])
mask = torch.stack([mask, mask, mask])

Using MPS (Metal Performance Shaders) device
Using MPS (Metal Performance Shaders) device


In [7]:
xx = model(x, mask)

In [8]:
torch.nn.functional.sigmoid(xx)

tensor([[0.7013],
        [0.3391],
        [0.5674]], device='mps:0', grad_fn=<SigmoidBackward0>)